In [1]:
import io
import requests
import docx

In [2]:
def clean_line(line):
    line = line.strip()
    line = line.strip('\uFEFF')
    return line

def get_font_size(paragraph):
    if paragraph.runs:
        for run in paragraph.runs:
            if run.font.size:
                return run.font.size.pt
    return None


def read_faq(file_id):
    url = f'https://docs.google.com/document/d/{file_id}/export?format=docx'
    
    response = requests.get(url)
    response.raise_for_status()
    
    with io.BytesIO(response.content) as f_in:
        doc = docx.Document(f_in)

    questions = []

    section_heading_style = 'heading 1'
    question_heading_style = 'heading 2'
        
    heading_id = ''
    section_title = ''
    question_title = ''
    answer_text_so_far = ''
    
    for p in doc.paragraphs:
        style = p.style.name.lower()
        p_text = clean_line(p.text)
        p_text_size = get_font_size(p)
    
        if len(p_text) == 0:
            continue

        if p_text_size == 8.5:
            continue
    
        if style == section_heading_style:
            section_title = p_text
            continue
    
        if style == question_heading_style:
            answer_text_so_far = answer_text_so_far.strip()
            if answer_text_so_far != '' and section_title != '' and question_title != '':
                questions.append({
                    'text': answer_text_so_far,
                    'section': section_title,
                    'question': question_title,
                })
                answer_text_so_far = ''
    
            question_title = p_text
            continue
        
        answer_text_so_far += '\n' + p_text
    
    answer_text_so_far = answer_text_so_far.strip()
    if answer_text_so_far != '' and section_title != '' and question_title != '':
        questions.append({
            'text': answer_text_so_far,
            'section': section_title,
            'question': question_title,
        })

    return questions


In [3]:
faq_documents = {
    'data-engineering-zoomcamp': '19bnYs80DwuUimHM65UV3sylsCn2j1vziPOwzBwQrebw',
    'machine-learning-zoomcamp': '1LpPanc33QJJ6BSsyxVg-pWNMplal84TdZtq10naIhD8',
    'mlops-zoomcamp': '12TlBfhIiKtyBv8RnsoJR6F72bkPDGEvPOItJIxaEzE0',
}

In [4]:
documents = []

for course, file_id in faq_documents.items():
    print(course)
    course_documents = read_faq(file_id)
    documents.append({'course': course, 'documents': course_documents})
    

data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp


In [5]:
import json

In [6]:
with open('documents.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [12]:
!head -n 20 documents.json

[
  {
    "course": "data-engineering-zoomcamp",
    "documents": [
      {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?"
      },
      {
        "text": "GitHub - DataTalksClub/data-engineering-zoomcamp#prerequisites",
        "section": "General course-related questions",
        "question": "Course - What are the prerequisites for this course?"
      },
      {
        "text": "Yes, even if you don't register, you're still eligibl